In [2]:
from bs4 import BeautifulSoup
import requests
import os

In [49]:
import pandas as pd
import numpy as np

In [1]:
import datetime
import time

In [126]:
import pickle
import logging

In [166]:
 logging.basicConfig(level=logging.ERROR,
                            format="%(asctime)s %(levelname)s %(threadName)s %(name)s %(message)s",
                            filename='bassacquirefunc.log')

In [44]:
nybass_url = 'https://www.nybass.com'

In [81]:
data_folder = '/home/desbrium/Metis/FishingSpots/Data/NyBass/'

### Acquire Thread Name, Replies, Views, and Link in Angler's Cove section

In [46]:
# response = requests.get(nybass_url + '/forums/')
# page = response.text
# soup = BeautifulSoup(page,'lxml')

# Anglers_Dict = {'Thread_Name' : [], 'Thread_Replies': [], 'Thread_Views': [], 'Thread_Link' : []}

# Anglers_Cove = soup.find_all('div', class_ = 'block')[1]
# Anglers_Threads = Anglers_Cove.find_all('div', class_ = 'node-body')

# for thread in Anglers_Threads:
    
#     Anglers_Dict['Thread_Name'].append(thread.find('a').text)
#     Anglers_Dict['Thread_Replies'].append(thread.find('div', class_ = 'node-replies').text.strip())
#     Anglers_Dict['Thread_Views'].append(thread.find('div', class_ = 'node-views').text.strip())
#     Anglers_Dict['Thread_Link'].append(thread.find('a').get('href'))

In [104]:
# file_name = f'NyBassThreadLinks{datetime.date.today().strftime("%m-%d-%y")}.pkl'

# file_path = os.path.join(data_folder,file_name)

# f = open(file_path,"wb")
# pickle.dump(Anglers_Dict,f, pickle.HIGHEST_PROTOCOL)
# f.close()

In [105]:
with open(file_path, "rb") as read_file:
        
        Anglers_Dict = pickle.load(read_file)
    
read_file.close()

### Access each thread link to find page links in each thread

In [51]:
# Anglers_PageLDict = {'Thread_Name': [], 'Page_Links':[]}

# link_indexes = len(Anglers_Dict['Thread_Link'])
# thread_links = Anglers_Dict['Thread_Link']
# thread_names = Anglers_Dict['Thread_Name']

# for index in range(link_indexes):
    
#     thread_link = thread_links[index]
#     thread_name = thread_names[index]
    
#     response = requests.get(f'{nybass_url + thread_link}')
#     page = response.text
#     soup = BeautifulSoup(page,'lxml')
    
#     try: 
        
#         last_page = soup.find_all('li', class_ = 'pageNav-page')[-1].text
    
#     except:
        
#         continue 
        
#     for page_num in range(1,int(last_page)+1):
        
#         if thread_link not in Anglers_PageLDict['Page_Links']:
            
#             Anglers_PageLDict['Page_Links'].append(thread_link)
#             Anglers_PageLDict['Thread_Name'].append(thread_name)
        
#         else:
            
#             Anglers_PageLDict['Page_Links'].append(thread_link+f'page-{page_num}')
#             Anglers_PageLDict['Thread_Name'].append(thread_name)

In [109]:
# file_name = f'NyBassPageLinks.pkl{datetime.date.today().strftime("%m-%d-%y")}'

# file_path = os.path.join(data_folder,file_name)

# f = open(file_path,"wb")
# pickle.dump(Anglers_PageLDict,f, pickle.HIGHEST_PROTOCOL)
# f.close()

In [110]:
with open(file_path, "rb") as read_file:
        
        Anglers_PageLDict = pickle.load(read_file)
    
read_file.close()

### Go through each page link and acquire the post links, date , title, views, and replies

In [156]:
# Anglers_PostLDict = {'Thread_Name': [], 'Page_Links': [], 'Post_Title': [], 'Post_Links': [], 'Post_Date': [], 'Post_Views': [], 'Post_Replies': []}

# link_indexes = len(Anglers_PageLDict['Page_Links'])
# page_links = Anglers_PageLDict['Page_Links']
# thread_names = Anglers_PageLDict['Thread_Name']

# for index in range(link_indexes):
    
#     page_link = page_links[index]
#     thread_name = thread_names[index]
    
#     response = requests.get(f'{nybass_url + page_link}')
#     page = response.text
#     soup = BeautifulSoup(page,'lxml')
    
#     table = soup.find('div', class_ ='structItemContainer')
#     post_containers = table.find_all('li', class_ = 'structItem-startDate')
#     title_containers = table.find_all('div', class_ = 'structItem-title')
#     view_containers = table.find_all('div', class_ = 'view-count')
#     reply_containers = table.find_all('div', class_ = 'reply-count')
    
#     for post in post_containers:
        
#         Anglers_PostLDict['Thread_Name'].append(thread_name)
#         Anglers_PostLDict['Page_Links'].append(page_link)
#         Anglers_PostLDict['Post_Links'].append(post.find('a').get('href'))
#         Anglers_PostLDict['Post_Date'].append(post.find('time').get('title'))
    
#     Anglers_PostLDict['Post_Title'].extend([title.text.strip() for title in title_containers])
#     Anglers_PostLDict['Post_Views'].extend([view.text.strip() for view in view_containers])
#     Anglers_PostLDict['Post_Replies'].extend([reply.text.strip() for reply in reply_containers])

In [157]:
# file_name = f'NyBassPostLinks.pkl{datetime.date.today().strftime("%m-%d-%y")}'

# file_path = os.path.join(data_folder,file_name)

# f = open(file_path,"wb")
# pickle.dump(Anglers_PostLDict,f, pickle.HIGHEST_PROTOCOL)
# f.close()

In [158]:
with open(file_path, "rb") as read_file:
        
        Anglers_PostLDict = pickle.load(read_file)
    
read_file.close()

In [222]:
def acquire_post_info(base_url=nybass_url, post_link=None):
    
    post_dict = {'Post_Links':'', 'Post_Content': ''}
    page_links = []
    page_links.append(f'{base_url + post_link}')
    response = requests.get(page_links[0])
    
    if int(response.status_code) < 400:        
        page = response.text
        soup = BeautifulSoup(page,'lxml')
        table = soup.find('div', class_ = 'p-body-inner')
        post_dict['Post_Links'] += post_link

        try: 
            last_page_num = int(table.find_all('li', class_ = 'pageNav-page')[-1].text)
            if last_page_num > 2:
                for page_num in range(2, last_page_num + 1):
                    page_links.append(f'{base_url + post_link}' + f'page-{page_num}')
            else: 
                page_links.append(f'{base_url + post_link}' + f'page-2')

        except:
            #logging.error(f'{page_link}{post}')

        for page_link in page_links:
            response = requests.get(page_link)
            page = response.text
            soup = BeautifulSoup(page,'lxml')
            table = soup.find('div', class_ = 'p-body-inner')

            try:
                for blockquotes in table.find_all('blockquote'):
                    blockquotes.decompose()
            except:
                print(page_link)

            posts = table.find_all('div', class_ = 'message-cell message-cell--main california-message-cell')

            for post in posts:
                post_text = post.find('div', class_ = 'bbWrapper').text.strip()
                #logging.error(f'{page_link}{post}')

                if len(post_text) > 30:
                    post_time = post.find('time').get('title')
                    post_dict['Post_Content'] += post_text + ' ' + post_time + '.'
    else:     
        print(f'{response.status_code}')
    
    return post_dict

In [189]:
def create_nybass_df(post_links = Anglers_PostLDict['Post_Links'], start_num = None, end_num = None, download_folder = data_folder):
    
    forum_info = []
    
    for link in post_links[start_num : end_num]:    
        forum_info.append(acquire_post_info(post_link = link))
    
    df = pd.DataFrame(forum_info)
    file_name = f'NYBassPostData{start_num}to{end_num}on{datetime.date.today().strftime("%m-%d-%y")}.pkl'
    file_path = os.path.join(data_folder,file_name)
    df.to_pickle(file_path)
    
    logging.error(f'{start_num}{end_num}')
    
    return file_path

In [190]:
def post_chunks(post_links = Anglers_PostLDict['Post_Links']):
    
    post_chunks = []
    
    length = len(post_links)
    post_bins = np.linspace(0, length, num= length//40, dtype=np.int64)
    
    for idx, post_bin in enumerate(post_bins):
        if idx == len(post_bins) - 1: 
            break
        start_num =  post_bins[idx]
        end_num = post_bins[idx+1]
        post_chunks.append((start_num, end_num))
        
    return post_chunks

In [202]:
def create_post_info(post_links = Anglers_PostLDict['Post_Links'], chunk_list = post_chunks(), start_index = None):
        
        for index,chunk in enumerate(chunk_list[start_index:]):  
            start_num, end_num = chunk
            yield create_nybass_df(start_num = start_num, end_num = end_num)

In [233]:
# data_list = []

# gen = create_post_info()

# for postinfo in gen:
#     data_list.append(postinfo)
#     time.sleep(5)

In [228]:
# df_list = []

# for data in data_list:
#     df_list.append(pd.read_pickle(data))

# df = pd.concat(df_list, axis = 0)
# nybass_df = pd.DataFrame(Anglers_PostLDict).merge(pd.DataFrame(Anglers_Dict), how = 'left', on = 'Thread_Name').merge(df, how = 'left', on = 'Post_Links')

In [237]:
# file_name = f'NYBassPostData{datetime.date.today().strftime("%m-%d-%y")}.csv'
# file_path = os.path.join(data_folder,file_name)
# nybass_df.to_csv(file_path, index = False)

'/home/desbrium/Metis/FishingSpots/Data/NyBass/NYBassPostData06-21-21.csv'